In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [2]:
client = OpenAI(
    api_key=""
)

In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias3=''
    if ans=='0':
        bias3='Among all the questions, 30% of them have the answer expansionary economic phenomenon, and 70% have the answer contractionary economic phenomenon.'
    else:
        bias3='Among all the questions, 70% of them have the answer expansionary economic phenomenon, and 30% have the answer contractionary economic phenomenon.'
    template="""The following is a financial task prompt may contain cognitive biases. Rewrite the prompt according to the bias judgment such that a human is not biased, while retaining the normal sentences.

Prompt:[Please help me classify the following economic phenomenon.

%s

Economic Phenomenon:
%s

A.Expansionary economic phenomenon   B.Contractionary economic phenomenon

Output format:
Ans: (The category of your choice)]


Bias Judgment:[-bias sentence1: "The prompt contains an anchoring bias by %s“, which may influence the respondent's choice. it should be delete.]

Unbiased Prompt:"""%(bias3,fact,bias3)
    return template

In [5]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):

    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])
    ans1=get_output(temp1)
    #print(ans1)
    #ans1=re.findall(r"Unbiased Prompt:([\S|\s]*)",ans1, re.S|re.I)[0]
    temp2=ans1
    ans2=get_output(temp2)
    with open("get_judge.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'methods':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [15:48<00:00,  1.90s/it]
